<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/reinforce_reward_system_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

In [5]:
tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-CoLA")
model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-CoLA")
cola_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

sentiment_pipeline = pipeline('text-classification', 'cardiffnlp/twitter-roberta-base-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def get_cola_scores(texts):
  scores = []
  results = cola_pipeline(texts, function_to_apply='none', top_k=None)
  for result in results:
    for label in result:
      if label['label'] == 'LABEL_1':
        scores.append(label['score'])
  return scores

def get_sentiment_scores(texts):
  scores = []
  results = sentiment_pipeline(texts, function_to_apply='none', top_k=None)
  for result in results:
    for label in result:
      if label['label'] == 'LABEL_1':
        scores.append(label['score'])
  return scores

In [11]:
texts = ['The Eiffel Tower in Paris is the tallest structure in the world, with a height of 1,063 meters',
         'This is a bad book',
         'This is a bad books']

In [20]:
cola_scores = get_cola_scores(texts)
sentiment_scores = get_sentiment_scores(texts)
print("cola : ", cola_scores)
print("sentiment : ", sentiment_scores)

cola :  [1.9864197969436646, 1.8140456676483154, -0.5497040748596191]
sentiment :  [0.7717742919921875, -0.48836588859558105, -0.26357436180114746]


In [16]:
transposed_lists = zip(cola_scores, sentiment_scores)

rewards = [1 * values[0] + 0.5 * values[1] for values in transposed_lists]
reward = [torch.tensor([_]) for _ in rewards]

In [17]:
print(reward)

[tensor([2.3723]), tensor([1.5699]), tensor([-0.6815])]
